In [1]:
import os
import sys
sys.path.insert(0,'..')
from nrel_xstock.database import ResstockDatabase, SQLiteDatabase
from nrel_xstock.simulate import CityLearnSimulator, OpenStudioModelEditor

In [2]:
database_filepath = 'database.sql'
resstock_database = ResstockDatabase(filepath=database_filepath)

# Build Database
***

In [3]:
# filters = {
#     'in.geometry_building_type_recs':['Single-Family Detached'],
#     'in.vacancy_status':['Occupied'],
#     'in.resstock_county_id':['TX, Travis County']
# }
# kwargs = {
#     'dataset_type':'resstock',
#     'weather_data':'tmy3',
#     'year_of_publication':2021,
#     'release':1,
#     'filters':filters
# }
# resstock_database.build(overwrite=True)
# resstock_database.insert_dataset(**kwargs)

# Simulation
***

In [4]:
# get model data
idd_filepath = '/Applications/EnergyPlus-9-6-0/PreProcess/IDFVersionUpdater/V9-6-0-Energy+.idd'
bldg_id = '40'
output_directory = f'{bldg_id}_simulation'
schedules_filepath = 'schedules.csv'
sim_data = resstock_database.query_table(f"SELECT metadata_id, osm, epw, ddy FROM building_energy_performance_simulation_input WHERE bldg_id = {bldg_id}")
metadata_id, osm, epw, ddy = sim_data.to_records(index=False)[0]
schedule = resstock_database.query_table(f"SELECT * FROM schedule WHERE metadata_id = {metadata_id}")
schedule = schedule.drop(columns=['metadata_id','day','hour','minute',])
schedule.to_csv(schedules_filepath,index=False)

# preprocess osm as needed and translate to idf
osm_editor = OpenStudioModelEditor(osm)
osm_editor.use_ideal_loads_air_system()
idf = osm_editor.forward_translate()
simulator = CityLearnSimulator(idd_filepath,idf,epw,simulation_id=bldg_id,output_directory=output_directory)
simulator.preprocess()

# simulate
simulator.simulate()
os.remove(schedules_filepath)
simulation_result = simulator.get_simulation_result('misc/citylearn_simulation_result.sql')
display(simulation_result.head())

Could not find platform independent libraries <prefix>
Could not find platform dependent libraries <exec_prefix>
Consider setting $PYTHONHOME to <prefix>[:<exec_prefix>]
EnergyPlus Completed Successfully.


,Month,Hour,Day Type,Daylight Savings Status,Indoor Temperature [C],Average Unmet Cooling Setpoint Difference [C],Average Unmet Heating Setpoint Difference [C],Indoor Relative Humidity [%],Equipment Electric Power [kWh],DHW Heating [kWh],Cooling Load [kWh],Heating Load [kWh]
0,1,1,1,0,22.761658,1.127231,2.206102,30.646228,0.338619,0.841175,0.0,0.0
1,1,2,1,0,22.452145,1.436744,1.896590,31.245663,0.254371,0.772947,0.0,0.0
2,1,3,1,0,22.107612,1.781277,1.552056,31.970667,0.188268,1.317160,0.0,0.0
3,1,4,1,0,21.800468,2.088421,1.244912,32.603083,0.186285,0.761560,0.0,0.0
4,1,5,1,0,21.395272,2.493617,0.839717,33.490149,0.216485,0.711717,0.0,0.0
